# Auto-encoder and PCA comparison using Little House Dataset

### Importing required libraries

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl

from keras.layers import Dense, Flatten, Reshape, Input, InputLayer, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model

import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn import metrics

from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy import interpolate

from livelossplot import PlotLossesKeras

### Custom functions

In [ ]:
def centroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return sum_x/length, sum_y/length

In [ ]:
def hull_interpolation(x_hull, y_hull):
    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))
    spline, u = interpolate.splprep([x_hull, y_hull], 
                                    u=dist_along, s=0, per=1)
    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
    interp_x, interp_y = interpolate.splev(interp_d, spline)
    return [interp_x, interp_y]

In [ ]:
def plot3clusters(X, y, title, vtitle, target_names):
    plt.figure()
    colors = ['r', 'g', 'k', 'b']
    lw = 2

    for color, i, target_name in zip(colors, [0, 1, 2, 3], target_names):
        plt.scatter(X[y == i, 0], X[y == i, 1], color=color, alpha=1., lw=lw, label=target_name)
        points = np.take(X[y == i], [0,1], 1)
        cent = centroidnp(points)
        plt.plot(cent[0], cent[1], alpha=0.95, c=color, marker='D')
        for n in range(0,points.shape[0]):
            x_point = [points[n, 0], cent[0]]
            y_point = [points[n, 1], cent[1]]
            plt.plot(x_point, y_point, alpha=0.4, c=color)
    
    plt.legend(loc='best', shadow=False, scatterpoints=1)
    plt.title(title)  
    plt.xlabel(vtitle + "1")
    plt.ylabel(vtitle + "2")
    plt.show()

In [ ]:
def plot4clusters(X, y, title, vtitle, target_names):
    plt.figure()
    colors = ['r', 'g', 'c', 'k', 'b']
    # colors = ['k', 'k', 'k', 'k', 'k']
    lw = 2

    for color, i, target_name in zip(colors, [0, 1, 2, 3, 4], target_names):
        plt.scatter(X[y == i, 0], X[y == i, 1], color=color, alpha=1., lw=lw, label=target_name)
        points = np.take(X[y == i], [0,1], 1)
        cent = centroidnp(points)
        plt.plot(cent[0], cent[1], alpha=0.95, c=color, marker='D')
        for n in range(0,points.shape[0]):
            x_point = [points[n, 0], cent[0]]
            y_point = [points[n, 1], cent[1]]
            plt.plot(x_point, y_point, alpha=0.4, c=color)
    
    plt.legend(loc='best', shadow=False, scatterpoints=1)
    plt.title(title)  
    plt.xlabel(vtitle + "1")
    plt.ylabel(vtitle + "2")
    plt.show()

In [ ]:
def plot5clusters(X, y, title, vtitle, target_names):
    plt.figure()
    colors = ['r', 'g', 'k', 'c', 'y', 'b']
    # colors = ['k', 'k', 'k', 'k', 'k', 'k']
    lw = 2

    for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5], target_names):
        plt.scatter(X[y == i, 0], X[y == i, 1], color=color, alpha=1., lw=lw, label=target_name)
        points = np.take(X[y == i], [0,1], 1)
        cent = centroidnp(points)
        plt.plot(cent[0], cent[1], alpha=0.95, c=color, marker='D')
        for n in range(0,points.shape[0]):
            x_point = [points[n, 0], cent[0]]
            y_point = [points[n, 1], cent[1]]
            plt.plot(x_point, y_point, alpha=0.4, c=color)
    
    plt.legend(loc='best', shadow=False, scatterpoints=1)
    plt.title(title)  
    plt.xlabel(vtitle + "1")
    plt.ylabel(vtitle + "2")
    plt.show()

## Loading dataset

In [ ]:
stats = np.load('edt_statistics.npy')
stats = stats.reshape(53,500,500)
num_files = np.shape(stats)[0]
stats = stats.reshape(num_files,-1)

stats_scaled = StandardScaler().fit_transform(stats)
stats_scaled = stats_scaled.reshape(53,500,500,1)

### Label Creation

In [ ]:
label_sample = np.zeros((num_files))
label_bld_plt = np.zeros((num_files))

label_sample[0:5] = 0
label_sample[16:19] = 0
label_sample[38:43] = 0
label_sample[5:10] = 1
label_sample[28:33] = 1
label_sample[43:48] = 1
label_sample[33:38] = 2
label_sample[10:15] = 2
label_sample[48:53] = 2
label_sample[19:24] = 3
label_sample[24:28] = 4
label_sample[15:16] = 5

label_bld_plt[0:15] = 0
label_bld_plt[16:19] = 1
label_bld_plt[19:28] = 2
label_bld_plt[28:38] = 1
label_bld_plt[38:53] = 2
label_bld_plt[15:16] = 3

sample_name = ['Sample 11', 'Sample 6', 'Sample 7', 'Sample 13', 'Sample 1', ' ']
bld_plt_name = ['NP220302-1', 'NP220322-2', 'NP220404-1', ' ']

# Auto Encoder

### Autoencoder definition

In [ ]:
def build_linear_autoencoder(inp_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(inp_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size, activation='linear', kernel_initializer='glorot_uniform'))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size)))
    decoder.add(Dense(np.prod(inp_shape), activation='linear', kernel_initializer='glorot_uniform')) 
    decoder.add(Reshape(inp_shape))

    return encoder, decoder

In [ ]:
def build_sigmoid_autoencoder(inp_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(inp_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size, activation='sigmoid', kernel_initializer='glorot_uniform'))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size)))
    decoder.add(Dense(np.prod(inp_shape), activation='sigmoid', kernel_initializer='glorot_uniform')) 
    decoder.add(Reshape(inp_shape))

    return encoder, decoder

In [ ]:
def build_relu_autoencoder(inp_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(inp_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size, activation='relu', kernel_initializer='glorot_uniform'))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size)))
    decoder.add(Dense(np.prod(inp_shape), activation='sigmoid', kernel_initializer='glorot_uniform')) 
    decoder.add(Reshape(inp_shape))

    return encoder, decoder

In [ ]:
def build_var_autoencoder(inp_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(inp_shape))
    encoder.add(Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'))
    encoder.add(Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'))
    encoder.add(Conv2D(filters=128, kernel_size=3, strides=2, activation='relu'))
    encoder.add(Flatten())
    encoder.add(Dense(code_size))#, activation='relu', kernel_initializer='glorot_uniform'))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size,)))
    # decoder.add(Dense(np.prod(inp_shape), activation='sigmoid', kernel_initializer='glorot_uniform'))
    decoder.add(Dense(np.prod(inp_shape), activation='relu'))#, kernel_initializer='glorot_uniform')) 
    decoder.add(Reshape(target_shape=(500, 500, 1)))
    decoder.add(Conv2DTranspose(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
    decoder.add(Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
    decoder.add(Conv2DTranspose(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
    decoder.add(Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same'))
    # decoder.add(Reshape(inp_shape))

    return encoder, decoder

### Autoencoder Setup

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
input_size = stats_scaled.shape[1:]
encoder_vae, decoder_vae = build_var_autoencoder(input_size, 256)

inp = Input(input_size)
code_vae = encoder_vae(inp)
reconstruct = decoder_vae(code_vae)

autoencoder_vae = Model(inp,reconstruct)
autoencoder_vae.compile(optimizer='Adam', loss='KLDivergence')

print(autoencoder_vae.summary())

In [ ]:
history_vae = autoencoder_vae.fit(stats_scaled, stats_scaled, batch_size = 0, epochs = 500, validation_split=0.1, callbacks = [], verbose = 0)

plt.plot(history_vae.history['loss'])
plt.plot(history_vae.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
encoded_data_vae = encoder_vae.predict(stats_scaled)

### Plotting VAE results

In [ ]:
plot5clusters(encoded_data_vae[:,:3], label_sample, 'VAE (sample)', 'AE', sample_name)
plot4clusters(encoded_data_vae[:,:3], label_bld_plt, 'VAE (build plate)', 'AE', bld_plt_name)

## PCA on latent variables

In [ ]:
pca = PCA(n_components = 10)
pca_transformed = pca.fit_transform(stats)

In [ ]:
plot5clusters(pca_transformed[:,:3], label_sample, 'VAE (sample)', 'AE', sample_name)
plot4clusters(pca_transformed[:,:3], label_bld_plt, 'VAE (build plate)', 'AE', bld_plt_name)

## Clustering Metrics

**homogeneity** Each cluster contains only members of a single class  

**completeness** All members of a given class are assigned to the same cluster  

**V-measure** Harmonic mean of homogeneity and completeness  

$v = \frac{(1+\beta) \times homogeneity \times completeness}{\beta \times (homogeneity + completeness)}$

**Rand Index** Similarity of the prediction with the ground truth  

**Mutual Information** Function that measures the agreement of the predicted label with the ground truth

**Silhouette Coefficient** If ground truth labels are not known, this coefficient relates to a model with better defined clusters. 

In [ ]:
labels_true = label_sample 
titles = ['Linear AE', 'Non-Linear (sigmoid) AE', 'Non-Linear (relu)']
for n_clusters_ in [2:4]:
      estimators = [
                    ('AE linear' , KMeans(n_clusters=n_clusters_), encoded_data_LE),
                    ('AE Non-linear (sigmoid)' , KMeans(n_clusters=n_clusters_), encoded_data_sig),
                    ('AE Non-linear (relu)' , KMeans(n_clusters=n_clusters_), encoded_data_rel)]

      print('Number of clusters: %d' % n_clusters_)
      for name, est, data in estimators:
            X = data
            est.fit(X)
            labels = est.labels_
            print(name,':')
      #   print(labels[:]) 
            print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
            print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
            print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
            print("Adjusted Rand Index: %0.3f"
                  % metrics.adjusted_rand_score(labels_true, labels))
            print("Adjusted Mutual Information: %0.3f"
                  % metrics.adjusted_mutual_info_score(labels_true, labels))
            print("Silhouette Coefficient: %0.3f"
                  % metrics.silhouette_score(X, labels))
            print()
      print()
      print('----------------------------------------------------------------------------------')
      print()

In [ ]:
labels_true = label_sample 
titles = ['PCA', 'Linear AE', 'Non-Linear (sigmoid) AE', 'Non-Linear (relu)']
for n_clusters_ in [3]:
      estimators = [('PCA'    , KMeans(n_clusters=n_clusters_), pca_transformed),
                    ('AE linear' , KMeans(n_clusters=n_clusters_), encoded_data_LE),
                    ('AE Non-linear (sigmoid)' , KMeans(n_clusters=n_clusters_), encoded_data_sig),
                    ('AE Non-linear (relu)' , KMeans(n_clusters=n_clusters_), encoded_data_rel)]

      print('Number of clusters: %d' % n_clusters_)
      for name, est, data in estimators:
            X = data
            est.fit(X)
            labels = est.labels_
            print(name,':')
      #   print(labels[:]) 
            print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
            print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
            print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
            print("Adjusted Rand Index: %0.3f"
                  % metrics.adjusted_rand_score(labels_true, labels))
            print("Adjusted Mutual Information: %0.3f"
                  % metrics.adjusted_mutual_info_score(labels_true, labels))
            print("Silhouette Coefficient: %0.3f"
                  % metrics.silhouette_score(X, labels))
            print()
      print()
      print('----------------------------------------------------------------------------------')
      print()

In [ ]:
labels_true = label_sample 
titles = ['PCA', 'Linear AE', 'Non-Linear (sigmoid) AE', 'Non-Linear (relu)']
for n_clusters_ in [3]:
      estimators = [('PCA'    , KMeans(n_clusters=n_clusters_), pca_transformed),
                    ('AE linear' , KMeans(n_clusters=n_clusters_), encoded_data_LE),
                    ('AE Non-linear (sigmoid)' , KMeans(n_clusters=n_clusters_), encoded_data_sig),
                    ('AE Non-linear (relu)' , KMeans(n_clusters=n_clusters_), encoded_data_rel)]

    print('Number of clusters: %d' % n_clusters_)
    for name, est, data in estimators:
        X = data
        est.fit(X)
        labels = est.labels_
        print(name,':')
        print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
        print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
        print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
        print("Adjusted Rand Index: %0.3f"
                % metrics.adjusted_rand_score(labels_true, labels))
        print("Adjusted Mutual Information: %0.3f"
                % metrics.adjusted_mutual_info_score(labels_true, labels))
        print("Silhouette Coefficient: %0.3f"
                % metrics.silhouette_score(X, labels))
        print()
    print()
    print('----------------------------------------------------------------------------------')
    print()